In [ ]:
#@title نصب پکیج های مورد نیاز (1 دقیقه زمان میبره)
!pip install google-generativeai
!pip install pysrt
!pip install tqdm
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#@title فراخوانی کلید دسترسی
import google.generativeai as genai

# @markdown  **لطفاً کلید دسترسی خود را در کادر زیر وارد کنید (API KEY)**
GOOGLE_API_KEY = ""  #@param {type:"string"}

if not GOOGLE_API_KEY:
  print("هشدار: کلید API وارد نشده است.")
else:
  genai.configure(api_key=GOOGLE_API_KEY)
  print("کلید API با موفقیت پیکربندی شد.")

In [ ]:
#@title آپلود زیرنویس زبان اصلی
from google.colab import files
import io
import pysrt
import os

#@markdown #### فقط در صورتی که ادامه ترجمه را از بکاپ قبلی انجام می‌دهید، تیک بزنید
is_backup = False #@param {type:"boolean"}

uploaded = files.upload()
filename = list(uploaded.keys())[0]

if is_backup:
    # اگر فایل آپلود شده "backup.srt" نیست، سپس بکاپ قبلی را حذف کن
    if filename != "backup.srt":
        if os.path.exists("backup.srt"):
            os.remove("backup.srt")
        os.rename(filename, "backup.srt")
        filename = "backup.srt"

subs = pysrt.open(filename)
print(f"\nتعداد کل دیالوگ‌ها: {len(subs)}")


In [ ]:

#@title ترجمه به فارسی عامیانه

import pysrt
import google.generativeai as genai
from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential
#@markdown ### `تنظیم زمان استراحت بین هر درخواست (ثانیه) ترجیحا 3 یا بیشتر`

SLEEP_TIME = 3 #@param {type:"slider", min:2, max:5, step:0.5}

#@markdown ####انتخاب مدل هوش مصنوعی
MODEL_NAME = "gemini-1.5-flash" #@param ["gemini-2.0-flash-thinking-exp-01-21", "gemini-1.5-flash"]

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def translate_subtitle_with_retry(text):
    model = genai.GenerativeModel(MODEL_NAME, safety_settings={
        genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
    })

    prompt = f"""متن زیر را به فارسی عامیانه و روان ترجمه کن. نکات مهم:

1. فقط ترجمه خالص و عامیانه، بدون هیچ توضیح اضافه حتی حق نداری به من بگی باشه و یا هر اضافه گویی
2. به شدت مهمه که تا معنای واقعی جملات رو درک نکنی حق ترجمه نداری
3. استفاده از زبان محاوره‌ای و روزمره
4. نباید ترجمه های مختلف رو بگی فقط مرتبط ترین ترجمه رو بدون توضیح اضافه بگو

متن برای ترجمه:
{text}"""

    response = model.generate_content(prompt)
    time.sleep(SLEEP_TIME)
    return response.text

# بارگذاری زیرنویس
subs = pysrt.open(filename)

# شروع ترجمه با نمایش پیشرفت
with tqdm(total=len(subs), desc="ترجمه زیرنویس") as pbar:
    for i, sub in enumerate(subs):
        try:
            sub.text = translate_subtitle_with_retry(sub.text)
            pbar.update(1)

            # ایجاد بکاپ هر 20 دیالوگ
            if (i + 1) % 20 == 0:
                subs.save("backup.srt", encoding='utf-8')
                print(f"\nبکاپ تا دیالوگ {i+1} ذخیره شد")

        except Exception as e:
            subs.save("backup.srt", encoding='utf-8')
            print(f"\nخطا در دیالوگ {i+1}. فایل بکاپ ذخیره شد")
            raise

# ذخیره نهایی
subs.save(filename.replace(".srt", "_fa.srt"), encoding='utf-8')
print("\nترجمه با موفقیت به پایان رسید!")

In [ ]:

#@title دانلود زیرنویس ترجمه شده
from google.colab import files

files.download(filename.replace(".srt", "_fa.srt"))

--------------------------------

<font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#0066cc">─</font>

In [ ]:

#@title ادامه ترجمه از آخرین بکاپ
import pysrt
import google.generativeai as genai
from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential

#@markdown تنظیم زمان استراحت بین هر درخواست (ثانیه)
SLEEP_TIME = 3 #@param {type:"slider", min:2, max:5, step:0.5}
#@markdown ####شروع ترجمه از دیالوگ شماره:
START_DIALOG = 0 #@param {type:"integer"}
#@markdown انتخاب مدل هوش مصنوعی
MODEL_NAME = "gemini-1.5-flash" #@param ["gemini-2.0-flash-thinking-exp-01-21", "gemini-1.5-flash"]

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def translate_subtitle_with_retry(text):
    model = genai.GenerativeModel(MODEL_NAME, safety_settings={
        genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: genai.types.HarmBlockThreshold.BLOCK_NONE,
        genai.types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
    })

    prompt = f"""متن زیر را به فارسی عامیانه و روان ترجمه کن. نکات مهم:

1. فقط ترجمه خالص و عامیانه، بدون هیچ توضیح اضافه حتی حق نداری به من بگی باشه و یا هر اضافه گویی
2. به شدت مهمه که تا معنای واقعی جملات رو درک نکنی حق ترجمه نداری
3. استفاده از زبان محاوره‌ای و روزمره
4. نباید ترجمه های مختلف رو بگی فقط مرتبط ترین ترجمه رو بدون توضیح اضافه بگو

متن برای ترجمه:
{text}"""

    response = model.generate_content(prompt)
    time.sleep(SLEEP_TIME)
    return response.text

# بارگذاری زیرنویس از فایل بکاپ
subs = pysrt.open("backup.srt")
print(f"تعداد کل دیالوگ‌ها: {len(subs)}")

# شروع ترجمه با نمایش پیشرفت
with tqdm(total=len(subs)-START_DIALOG+1, desc="ترجمه زیرنویس") as pbar:
    for i, sub in enumerate(subs[START_DIALOG-1:], start=START_DIALOG-1):
        try:
            sub.text = translate_subtitle_with_retry(sub.text)
            pbar.update(1)

            # ایجاد بکاپ هر 20 دیالوگ
            if (i + 1) % 20 == 0:
                subs.save("backup.srt", encoding='utf-8')
                print(f"\nبکاپ تا دیالوگ {i+1} ذخیره شد")

        except Exception as e:
            subs.save("backup.srt", encoding='utf-8')
            print(f"\nخطا در دیالوگ {i+1}. فایل بکاپ ذخیره شد")
            raise

# ذخیره نهایی
subs.save(filename.replace(".srt", "_fa.srt"), encoding='utf-8')
print("\nترجمه با موفقیت به پایان رسید!")

<font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font>

<table><tr><td bgcolor="#40E0D0">
<font color="black">این سلول فقط در صورتی قابل استفاده هست که شما قصد دارید<br>روزهای بعد ادامه ترجمه رو انجام بدین و به این فایل نیاز دارید</font>
</td></tr></table>

In [ ]:

#@title دانلود آخرین فایل بکاپ
from google.colab import files

files.download("backup.srt")